In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'model_comparison_results.xlsx'
df = pd.read_excel(file_path, sheet_name='Frame Results')

# Compare the values in the specified columns
condition = (df['yolov5l'] != df['yolov8l']) | (df['yolov5l'] != df['yolo11l']) | (df['yolov8l'] != df['yolo11l'])

# Extract the Frame values where the condition is met
frames_with_diff = df[condition]['Frame']

# Load the original Excel file and write the result to a new sheet
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    frames_with_diff.to_frame().to_excel(writer, sheet_name='Frames_with_diff', index=False)

print("Frames with differences have been saved in a new sheet.")


In [ ]:
import pandas as pd
import cv2
import os

# Step 1: Read the Excel file and extract the frame numbers
excel_file = 'model_comparison_results.xlsx'
sheet_name = 'Frames_with_diff'

# Load the data
df = pd.read_excel(excel_file, sheet_name=sheet_name)

# Extract the 'Frame' column values
frame_numbers = df['Frame'].tolist()

# Step 2: Open the video file
video_file = 'video.mp4'
cap = cv2.VideoCapture(video_file)

# Check if video is opened correctly
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    # Step 3: Create a new folder within the existing 'output' folder to store screenshots
    output_dir = 'output/screenshots'
    os.makedirs(output_dir, exist_ok=True)
    
    # Step 4: Loop through the frame numbers and extract screenshots
    for i, frame_number in enumerate(frame_numbers):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        
        if ret:
            screenshot_filename = f'{output_dir}/frame_{frame_number}.png'
            cv2.imwrite(screenshot_filename, frame)
            print(f"Saved screenshot for frame {frame_number}")
        else:
            print(f"Failed to capture frame {frame_number}")

    # Release the video capture object
    cap.release()


In [7]:
import cv2
from ultralytics import YOLO
import os

# Load the models from the /models folder
yolo_v5l = YOLO('models/yolov5l.pt')
yolo_v8l = YOLO('models/yolov8l.pt')
yolo_v11l = YOLO('models/yolo11l.pt')

# Path to the screenshot image
screenshot_path = 'output/screenshots/frame_361.png'

# Function to process the frame and count the people
def count_people_and_annotate(frame, model):
    # Run the model on the frame
    results = model(frame)

    # Extract the results
    people_count = 0
    annotated_frame = frame.copy()

    # Loop through each detection result
    for result in results[0].boxes:
        if result is not None and result.shape[0] > 0:
            # Extract coordinates and class_id
            xyxy = result.xyxy[0]  # Bounding box coordinates
            confidence = result.conf[0]  # Confidence score
            class_id = int(result.cls[0].item())  # Class ID

            if class_id == 0:  # Check if the class is 'person' (class_id 0 in YOLO)
                x1, y1, x2, y2 = map(int, xyxy)  # Get bounding box coordinates
                people_count += 1
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw bounding box

    return people_count, annotated_frame

# Read the image file
frame = cv2.imread(screenshot_path)

if frame is not None:
    # Get the count and annotated frames for each model
    count_v5l, annotated_v5l = count_people_and_annotate(frame, yolo_v5l)
    count_v8l, annotated_v8l = count_people_and_annotate(frame, yolo_v8l)
    count_v11l, annotated_v11l = count_people_and_annotate(frame, yolo_v11l)

    # Create a combined image (side by side)
    combined_image = cv2.hconcat([annotated_v5l, annotated_v8l, annotated_v11l])

    # Add text to the combined image showing the counts
    cv2.putText(combined_image, f'YOLOv5l: {count_v5l} people', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(combined_image, f'YOLOv8l: {count_v8l} people', (frame.shape[1] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(combined_image, f'YOLOv11l: {count_v11l} people', (2 * frame.shape[1] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Save the combined comparison image
    output_dir = 'output/combined_comparisons'
    os.makedirs(output_dir, exist_ok=True)
    screenshot_filename = f'{output_dir}/frame_361_comparison.png'
    cv2.imwrite(screenshot_filename, combined_image)
    print(f"Saved combined comparison for frame_361.png")
else:
    print(f"Failed to read {screenshot_path}")


PRO TIP 💡 Replace 'model=models/yolov5l.pt' with new 'model=models/yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 640x640 6 persons, 81.4ms
Speed: 2.8ms preprocess, 81.4ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 46.1ms
Speed: 1.4ms preprocess, 46.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 32.8ms
Speed: 1.6ms preprocess, 32.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
Saved combined comparison for frame_361.png


In [10]:
import cv2
import os
from ultralytics import YOLO

# Load the models from the /models folder
yolo_v5l = YOLO('models/yolov5l.pt')
yolo_v8l = YOLO('models/yolov8l.pt')
yolo_v11l = YOLO('models/yolo11l.pt')

# Path to the screenshots folder
screenshots_dir = 'output/screenshots'
output_dir = 'output/combined_comparisons'
os.makedirs(output_dir, exist_ok=True)

# Function to process the frame and count the people
def count_people_and_annotate(frame, model):
    # Run the model on the frame
    results = model(frame)

    # Extract the results
    people_count = 0
    annotated_frame = frame.copy()

    # Loop through each detection result
    for result in results[0].boxes:
        if result is not None and result.shape[0] > 0:
            # Extract coordinates and class_id
            xyxy = result.xyxy[0]  # Bounding box coordinates
            confidence = result.conf[0]  # Confidence score
            class_id = int(result.cls[0].item())  # Class ID

            if class_id == 0:  # Check if the class is 'person' (class_id 0 in YOLO)
                x1, y1, x2, y2 = map(int, xyxy)  # Get bounding box coordinates
                people_count += 1
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw bounding box

    return people_count, annotated_frame

# Get all .png files in the screenshots folder
screenshot_files = [f for f in os.listdir(screenshots_dir) if f.endswith('.png')]

# Iterate through each screenshot file
for screenshot_file in screenshot_files:
    screenshot_path = os.path.join(screenshots_dir, screenshot_file)
    frame = cv2.imread(screenshot_path)

    if frame is not None:
        # Get the count and annotated frames for each model
        count_v5l, annotated_v5l = count_people_and_annotate(frame, yolo_v5l)
        count_v8l, annotated_v8l = count_people_and_annotate(frame, yolo_v8l)
        count_v11l, annotated_v11l = count_people_and_annotate(frame, yolo_v11l)

        # Create a combined image (side by side)
        combined_image = cv2.hconcat([annotated_v5l, annotated_v8l, annotated_v11l])

        # Add text to the combined image showing the counts
        cv2.putText(combined_image, f'YOLOv5l: {count_v5l} people', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(combined_image, f'YOLOv8l: {count_v8l} people', (frame.shape[1] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(combined_image, f'YOLOv11l: {count_v11l} people', (2 * frame.shape[1] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Save the combined comparison image
        screenshot_filename = f'{output_dir}/{screenshot_file.replace(".png", "_comparison.png")}'
        cv2.imwrite(screenshot_filename, combined_image)
        print(f"Saved combined comparison for {screenshot_file}")
    else:
        print(f"Failed to read {screenshot_file}")


PRO TIP 💡 Replace 'model=models/yolov5l.pt' with new 'model=models/yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 640x640 7 persons, 84.2ms
Speed: 3.0ms preprocess, 84.2ms inference, 5.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 persons, 46.1ms
Speed: 1.5ms preprocess, 46.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 1 remote, 35.6ms
Speed: 1.5ms preprocess, 35.6ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)
Saved combined comparison for frame_1000.png

0: 640x640 7 persons, 1 remote, 41.1ms
Speed: 1.7ms preprocess, 41.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 45.1ms
Speed: 1.9ms preprocess, 45.1ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 36.4ms

In [1]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = "model_comparison_results.xlsx"
sheet_name = "Frame Results"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Extract relevant columns
true_values = df['X']
models = ['yolov9c', 'yolov8l', 'yolo11l']

# Initialize results
rmse_results = {}
accuracy_results = {}

for model in models:
    # Calculate RMSE
    rmse = np.sqrt(np.mean((df[model] - true_values) ** 2))
    rmse_results[model] = rmse
    
    # Calculate Accuracy
    accuracy = np.mean(df[model] == true_values) * 100
    accuracy_results[model] = accuracy

# Display results
print("RMSE Results:")
for model, rmse in rmse_results.items():
    print(f"{model}: {rmse:.4f}")

print("\nAccuracy Results:")
for model, accuracy in accuracy_results.items():
    print(f"{model}: {accuracy:.2f}%")


KeyError: 'X'